In [1]:
# GPU 옵션 켜져 있는지 확인할 것 (수정 - 노트설정 - 하드웨어설정 (GPU))

'''
1. import: 필요한 모듈 import
2. 전처리: 학습에 필요한 데이터 전처리를 수행합니다.
3. 모델링(model): 모델을 정의합니다.
4. 컴파일(compile): 모델을 생성합니다.
5. 학습 (fit): 모델을 학습시킵니다.
'''

'\n1. import: 필요한 모듈 import\n2. 전처리: 학습에 필요한 데이터 전처리를 수행합니다.\n3. 모델링(model): 모델을 정의합니다.\n4. 컴파일(compile): 모델을 생성합니다.\n5. 학습 (fit): 모델을 학습시킵니다.\n'

In [2]:
import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import ModelCheckpoint

In [3]:
# 시험에서 주어지는 데이터셋 로드 형태
# 예전 방식이므로 아래처러 주어지는 코드를 과감히 삭제 후, 아래 제공되는 방식으로 변경
# data = tfds.load("iris", split=tfds.Split.TRAIN.subsplit(tfds.percent[:80]))

In [4]:
# train_dataset와 valid_dataset을 만들고 80% : 20%로 분할합니다.
train_dataset = tfds.load('iris', split='train[:80%]')
valid_dataset = tfds.load('iris', split='train[80%:]')
# valid_dataset = tfds.load('iris', split='train[-20%:]') 가능

Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

0 examples [00:00, ? examples/s]

Shuffling and writing examples to /root/tensorflow_datasets/iris/2.0.0.incompleteY12WKQ/iris-train.tfrecord


  0%|          | 0/150 [00:00<?, ? examples/s]

Dataset iris downloaded and prepared to /root/tensorflow_datasets/iris/2.0.0. Subsequent calls will reuse this data.


In [5]:
# 먼저, 시험에서는 다음과 같은 전처리 요구 조건이 있습니다.
# label 값을 one-hot encoding 할 것
# feature (x), label (y)를 분할할 것

def preprocess(data):
  x = data['features']
  y = data['label']
  y = tf.one_hot(y, 3)
  return x, y

In [6]:
batch_size=10

In [7]:
train_data=train_dataset.map(preprocess).batch(batch_size)
valid_data=valid_dataset.map(preprocess).batch(batch_size)

In [8]:
model = tf.keras.models.Sequential([
    # input_shape는 X의 feature 갯수가 4개 이므로 (4, )로 지정합니다.
    Dense(512, activation='relu', input_shape=(4,)),
    Dense(256, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(32, activation='relu'),
    # Classification을 위한 Softmax, 클래스 갯수 = 3개
    Dense(3, activation='softmax'),
])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 512)               2560      
                                                                 
 dense_1 (Dense)             (None, 256)               131328    
                                                                 
 dense_2 (Dense)             (None, 128)               32896     
                                                                 
 dense_3 (Dense)             (None, 64)                8256      
                                                                 
 dense_4 (Dense)             (None, 32)                2080      
                                                                 
 dense_5 (Dense)             (None, 3)                 99        
                                                                 
Total params: 177,219
Trainable params: 177,219
Non-trai

In [9]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['acc'])

In [10]:
checkpoint_path = "my_checkpoint.ckpt"
checkpoint = ModelCheckpoint(filepath=checkpoint_path, 
                             save_weights_only=True, 
                             save_best_only=True, 
                             monitor='val_loss', 
                             verbose=1)

In [11]:
history = model.fit(train_data,
                    validation_data=(valid_data),
                    epochs=20,
                    callbacks=[checkpoint]
                   )

Epoch 1/20
 9/12 [=====================>........] - ETA: 0s - loss: 0.9975 - acc: 0.4778 
Epoch 00001: val_loss improved from inf to 0.66333, saving model to my_checkpoint.ckpt
12/12 [==============================] - 4s 33ms/step - loss: 0.9357 - acc: 0.5333 - val_loss: 0.6633 - val_acc: 0.6333
Epoch 2/20
11/12 [==========================>...] - ETA: 0s - loss: 0.5175 - acc: 0.8091
Epoch 00002: val_loss improved from 0.66333 to 0.42185, saving model to my_checkpoint.ckpt
12/12 [==============================] - 0s 16ms/step - loss: 0.5077 - acc: 0.8167 - val_loss: 0.4219 - val_acc: 0.8333
Epoch 3/20
11/12 [==========================>...] - ETA: 0s - loss: 0.3270 - acc: 0.8455
Epoch 00003: val_loss improved from 0.42185 to 0.33079, saving model to my_checkpoint.ckpt
12/12 [==============================] - 0s 18ms/step - loss: 0.3229 - acc: 0.8500 - val_loss: 0.3308 - val_acc: 0.8333
Epoch 4/20
11/12 [==========================>...] - ETA: 0s - loss: 0.2106 - acc: 0.9636
Epoch 00004: v

In [12]:
# checkpoint 를 저장한 파일명을 입력합니다.
# 학습이 완료된 후에는 반드시 load_weights를 해주어야 합니다. 그렇지 않으면, 열심히 ModelCheckpoint를 만든 의미가 없습니다.
model.load_weights(checkpoint_path)